In [21]:
from scipy import io
import numpy as np
import os
import tensorflow as tf
import tensorflow.keras.backend as K
import math
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

from TA_CSPNN import *

In [39]:
for file in os.listdir('matlab_data/'):
    if not file == "A01T.mat":
        continue
    print(file)
    
    data = io.matlab.loadmat("matlab_data/" + file)['data']
    labels = io.matlab.loadmat('true_labels/' + file)['classlabel']
    break
print(data.shape)
print(labels.shape)

num_samples = data.shape[2]

train_size = int(num_samples * .9) # 90/10 train validation split
val_size = num_samples - train_size
print(f"num train samples: {train_size}")
print(f"num val samples: {val_size}")

A01T.mat
(22, 250, 288)
(288, 1)
num train samples: 259
num val samples: 29


In [45]:
rand_idx = np.random.randint(0,288, num_samples)
train_size = 10
val_size = 5
train_idx = rand_idx[:train_size]
val_idx = rand_idx[train_size:train_size+val_size]

x_train = data[:,:,train_idx]
y_train = labels[train_idx]

x_val = data[:,:,val_idx]
y_val = labels[val_idx]
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")

x_train shape: (22, 250, 10)
y_train shape: (10, 1)
x_val shape: (22, 250, 5)
y_val shape: (5, 1)


In [46]:
x_train = np.reshape(x_train, (train_size, 1, 22, 250))
x_val = np.reshape(x_val, (val_size, 1, 22, 250))

y_train = to_categorical(y_train - 1)
y_val = to_categorical(y_val - 1)

print(f"x_train shape: {x_train.shape}")
print(f"x_val shape: {x_val.shape}")

x_train shape: (10, 1, 22, 250)
x_val shape: (5, 1, 22, 250)


In [50]:
y_train

array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [51]:
labels[train_idx]

array([[4],
       [3],
       [1],
       [1],
       [3],
       [2],
       [2],
       [3],
       [2],
       [2]], dtype=uint8)

In [47]:
class model_config(object):
    def __init__(self):
        self.channels = 22
        self.timesamples = 250
        self.timeKernelLen = 64
        self.num_classes = 4
        self.Ft = 8
        self.Fs = 2

In [48]:
config = model_config()
model = TA_CSPNN(config.num_classes, Channels=config.channels, Timesamples=config.timesamples,
                timeKernelLen = config.timeKernelLen, Ft=config.Ft, Fs=config.Fs)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

In [49]:
history = model.fit(x_train, y_train, 
                    epochs=500, 
                    validation_data=((x_val, y_val)), 
                    callbacks=[es],
                    verbose=2)

Train on 10 samples, validate on 5 samples
Epoch 1/500
10/10 - 0s - loss: 1.9954 - acc: 0.1000 - val_loss: 3.6405 - val_acc: 0.0000e+00
Epoch 2/500
10/10 - 0s - loss: 1.9359 - acc: 0.3000 - val_loss: 3.3434 - val_acc: 0.0000e+00
Epoch 3/500
10/10 - 0s - loss: 2.4843 - acc: 0.4000 - val_loss: 3.1186 - val_acc: 0.0000e+00
Epoch 4/500
10/10 - 0s - loss: 2.3617 - acc: 0.3000 - val_loss: 2.9307 - val_acc: 0.0000e+00
Epoch 5/500
10/10 - 0s - loss: 2.1562 - acc: 0.4000 - val_loss: 2.7697 - val_acc: 0.0000e+00
Epoch 6/500
10/10 - 0s - loss: 2.2903 - acc: 0.1000 - val_loss: 2.6363 - val_acc: 0.0000e+00
Epoch 7/500
10/10 - 0s - loss: 1.9194 - acc: 0.3000 - val_loss: 2.5244 - val_acc: 0.0000e+00
Epoch 8/500
10/10 - 0s - loss: 1.6939 - acc: 0.2000 - val_loss: 2.4267 - val_acc: 0.0000e+00
Epoch 9/500
10/10 - 0s - loss: 1.7152 - acc: 0.4000 - val_loss: 2.3462 - val_acc: 0.0000e+00
Epoch 10/500
10/10 - 0s - loss: 2.1407 - acc: 0.2000 - val_loss: 2.2765 - val_acc: 0.0000e+00
Epoch 11/500
10/10 - 0s - 

Epoch 90/500
10/10 - 0s - loss: 0.4197 - acc: 0.8000 - val_loss: 1.3866 - val_acc: 0.2000
Epoch 91/500
10/10 - 0s - loss: 0.7131 - acc: 0.8000 - val_loss: 1.3873 - val_acc: 0.2000
Epoch 92/500
10/10 - 0s - loss: 0.3799 - acc: 1.0000 - val_loss: 1.3880 - val_acc: 0.2000
Epoch 93/500
10/10 - 0s - loss: 0.3567 - acc: 0.9000 - val_loss: 1.3890 - val_acc: 0.0000e+00
Epoch 94/500
10/10 - 0s - loss: 0.6649 - acc: 0.8000 - val_loss: 1.3903 - val_acc: 0.0000e+00
Epoch 95/500
10/10 - 0s - loss: 0.2965 - acc: 0.9000 - val_loss: 1.3918 - val_acc: 0.0000e+00
Epoch 96/500
10/10 - 0s - loss: 0.5250 - acc: 0.8000 - val_loss: 1.3937 - val_acc: 0.0000e+00
Epoch 97/500
10/10 - 0s - loss: 0.3352 - acc: 1.0000 - val_loss: 1.3958 - val_acc: 0.0000e+00
Epoch 98/500
10/10 - 0s - loss: 0.3998 - acc: 1.0000 - val_loss: 1.3980 - val_acc: 0.0000e+00
Epoch 99/500
10/10 - 0s - loss: 0.4996 - acc: 0.9000 - val_loss: 1.4002 - val_acc: 0.0000e+00
Epoch 100/500
10/10 - 0s - loss: 0.3547 - acc: 0.9000 - val_loss: 1.4025

In [70]:
file = file.replace('T','E')
file

'A04E.mat'

In [71]:
test_data = io.matlab.loadmat("matlab_data/" + file)['data']
y_test = io.matlab.loadmat('true_labels/' + file)['classlabel']

In [72]:
x_test = np.reshape(test_data, (288,1,22,250))

In [73]:
y_pred = model.predict(x_test)

In [74]:
pred = np.argmax(y_pred, axis=1) + 1

In [75]:
acc = 0
for i in range(len(y_test)):
    if pred[i] == y_test[i]:
        acc += 1

In [76]:
acc / len(y_test)

0.2569444444444444